In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from scipy.stats import pearsonr
import statsmodels.api as sm
import statsmodels.regression.linear_model as linear_model
from statsmodels.stats.outliers_influence import variance_inflation_factor



In [85]:
power_stations_raw = pd.read_csv('Data/power_stations_clean.csv', delimiter=';', encoding='latin1' )

power_stations_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2287 entries, 0 to 2286
Data columns (total 23 columns):
 #   Column                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                --------------  ----- 
 0   MaStR-Nr. der Stromerzeugungseinheit                                                                                                  2287 non-null   object
 1   Anlagenbetreiber                                                                                                                      2025 non-null   object
 2   Anzeige-Name der Stromerzeugungseinheit                                                                                               2120 non-null   object
 3   PLZ der Einheit                                                               